In [6]:
import sys

import numpy as np
import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.helpers.io as io
import src.helpers.data as hd
import src.data.aggregators as agg
import src.data.selectors as sl
import src.data.make_data as md

fs = s3fs.S3FileSystem(profile="3di")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fp = 's3://3di-data-mdb/clean'
# fp = 's3://3di-project-eval/'
fs.ls(fp, refresh=True, detail=False)

['3di-data-mdb/clean/',
 '3di-data-mdb/clean/mdb_000.parquet',
 '3di-data-mdb/clean/mdb_111.parquet',
 '3di-data-mdb/clean/mdb_777.parquet',
 '3di-data-mdb/clean/mdb_X11.parquet',
 '3di-data-mdb/clean/mdb_X77.parquet',
 '3di-data-mdb/clean/mdb_costa.parquet',
 '3di-data-mdb/clean/pieces',
 '3di-data-mdb/clean/samples']

In [12]:
df = io.read_parquet('s3://3di-project-eval/eval.parquet')
hd.inspect(df)

shape: (166,249, 27), users: 5884


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,tt,month_spend,user_registration_date,age,is_female,region,is_urban,has_sa_account,generation,generation_code,new_loan,pct_credit
0,361990,2016-04,201604,80,4767.620117,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-6,1002.900024,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0
1,361990,2016-05,201605,156,4787.720215,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-5,1881.459961,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0


In [16]:
df.netflows.describe()

count    166249.000000
mean        -19.899535
std        2210.107260
min      -11895.204687
25%           0.000000
50%           0.000000
75%          70.000000
max       11802.080000
Name: netflows, dtype: float64

In [25]:
hd.winsorise(df.outflows, pct=0, how='upper').describe()

count    166249.000000
mean        868.928984
std        2767.891407
min           0.000000
25%           0.000000
50%           0.000000
75%         441.540009
max       20194.205859
Name: outflows, dtype: float64

## dev

In [3]:
fp = '~/tmp/mdb_111.parquet'
df = md.read_piece(fp)
hd.inspect(df)

Reading ~/tmp/mdb_111.parquet
Time for read_piece                    : 1.09 minutes
shape: (593,133, 35), users: 270


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01


In [24]:
df.rename(columns={'ym': 'ho'})

,user_id,ho,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,tt,month_spend,user_registration_date,age,is_female,region_name,is_urban,has_sa_account,generation,generation_code,new_loan,unemp_benefits,pct_credit
0,361990,2016-04,201604,80,4767.620117,4947.571289,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,0.000000,0,-6,1002.900024,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0,0.000000
1,361990,2016-05,201605,156,4787.720215,4947.571289,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,0.000000,0,-5,1881.459961,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0,0.000000
2,361990,2016-06,201606,212,8925.620117,4947.571289,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,0.000000,0,-4,2195.699951,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0,0.000000
3,361990,2016-07,201607,98,2193.500000,4947.571289,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,0.000000,0,-3,1046.599976,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0,0.000000
4,361990,2016-08,201608,246,11848.889648,4947.571289,40.0,40.0,0.0,0.000000,0.008085,0.008085,0,0.000000,0,-2,2512.080078,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36596,589670,2020-04,202004,67,10439.450195,1816.088745,50.0,0.0,50.0,0.027532,0.027532,0.000000,1,0.027532,0,-4,877.020020,2020-08-14,25.0,1.0,yorkshire and the humber,1.0,True,Millennials,3,0,0,0.068413
36597,589670,2020-05,202005,59,7098.330078,1816.088745,100.0,0.0,100.0,0.055063,0.055063,0.000000,1,0.055063,0,-3,1246.410034,2020-08-14,25.0,1.0,yorkshire and the humber,1.0,True,Millennials,3,0,0,0.000000
36598,589670,2020-06,202006,73,5017.620117,1816.088745,50.0,0.0,50.0,0.027532,0.027532,0.000000,1,0.027532,0,-2,1726.349976,2020-08-14,25.0,1.0,yorkshire and the humber,1.0,True,Millennials,3,0,0,0.000000
36599,589670,2020-07,202007,93,45461.019531,1816.088745,13300.0,200.0,10600.0,3.607487,7.279748,0.110127,1,7.213304,0,-1,1579.390015,2020-08-14,25.0,1.0,yorkshire and the humber,1.0,True,Millennials,3,0,0,0.038623


In [12]:
print(f'Size on disk: {279685493 / 1e6:.2f}MB')
print(f'Size in memory: {df.memory_usage(deep=True).sum() / 1e6:.2f}MB')

Size on disk: 279.69MB
Size in memory: 2866.65MB


In [ ]:
19162689843 * 10 / 1e9

191.62689843

In [ ]:
df.memory_usage(deep=True).sum()

1045493

In [ ]:
df.date.gt('2012-12-14')

0      False
1      False
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999     True
Name: date, Length: 1000, dtype: bool

In [ ]:
df.age.describe()

count    410666.000000
mean         38.406345
std          14.503452
min          17.000000
25%          30.000000
50%          36.000000
75%          45.000000
max        1500.000000
Name: age, dtype: float64

In [ ]:
dfa = hd.read_analysis_data('XX1')
hd.inspect(dfa)

Time for read_analysis_data            : 0.77 seconds
shape: (44,316, 26), users: 1615


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,tt,month_spend,age,is_female,is_urban,has_sa_account,generation,generation_code,new_loan,unemp_benefits,pct_credit
0,366461,2016-04,201604,41,3271.570068,1458.706665,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-9,1307.880005,31.0,1.0,1.0,True,Millennials,3,0,0,100.0
1,366461,2016-05,201605,35,3998.689941,1458.706665,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-8,936.640015,31.0,1.0,1.0,True,Millennials,3,0,0,100.0


In [ ]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44316 entries, 0 to 44315
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype    
---  ------             --------------  -----    
 0   user_id            44316 non-null  int64    
 1   ym                 44316 non-null  period[M]
 2   ymn                44316 non-null  int64    
 3   txns_count         44316 non-null  int64    
 4   txns_volume        44316 non-null  float32  
 5   month_income       44316 non-null  float32  
 6   inflows            44316 non-null  float64  
 7   outflows           44316 non-null  float64  
 8   netflows           44316 non-null  float64  
 9   netflows_norm      44316 non-null  float64  
 10  inflows_norm       44316 non-null  float64  
 11  outflows_norm      44316 non-null  float64  
 12  has_pos_netflows   44316 non-null  int64    
 13  pos_netflows_norm  44316 non-null  float64  
 14  t                  44316 non-null  int64    
 15  tt                 44316 non-null  i

In [ ]:
d = hd.read_analysis_data('111')
hd.inspect(d)


Time for read_analysis_data            : 0.40 seconds
shape: (363, 26), users: 14


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,tt,month_spend,age,is_female,is_urban,has_sa_account,generation,generation_code,new_loan,unemp_benefits,pct_credit
0,400111,2016-10,201610,45,52008.281250,2121.186768,16100.0,916.580017,15183.419922,7.157983,7.590091,0.432107,1,7.157983,0,-12,1063.319946,27.0,1.0,0.0,True,Millennials,3,0,0,0.0
1,400111,2016-11,201611,199,68185.671875,2121.186768,0.0,18400.000000,-18400.000000,-8.674390,0.000000,8.674390,0,-0.000000,0,-11,3985.760010,27.0,1.0,0.0,True,Millennials,3,0,0,0.0


## txn data dev

In [40]:
dft = hd.read_txn_data('X11')
hd.inspect(dft)


Time for read_txn_data                 : 3.26 minutes
shape: (6,653,551, 35), users: 2734


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,11,69.750000,david lloyd <mdbremoved>,david lloyd,spend,sports,2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80656,True,0.0,False,False,1.0,150.029999,e01015428,david lloyd,e02003207,bh15 4,south west,10k to 20k,hobbies,gym membership,no tag,gym membership,u,2012-01
1,2012-01-03,11,96.400002,sky digital xxxxxxxxxx9317,sky,spend,"entertainment, tv, media",2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80654,True,0.0,False,False,1.0,150.029999,e01015428,sky,e02003207,bh15 4,south west,10k to 20k,services,"entertainment, tv, media",no tag,media bundle,u,2012-01


In [49]:
k = agg.age(dft)
k

user_id  ym     
11       2012-01    56.0
         2012-02    56.0
         2012-03    56.0
         2012-04    56.0
         2012-05    56.0
                    ... 
589711   2020-04    42.0
         2020-05    42.0
         2020-06    42.0
         2020-07    42.0
         2020-08    42.0
Name: age, Length: 79414, dtype: float64

In [50]:
k.describe()

count    70574.000000
mean        33.931136
std         11.196534
min         14.000000
25%         26.000000
50%         32.000000
75%         40.000000
max        134.000000
Name: age, dtype: float64

In [52]:
dft.birth_year.describe().round(0)

count    5902303.0
mean        1982.0
std           11.0
min         1880.0
25%         1976.0
50%         1984.0
75%         1990.0
max         2002.0
Name: birth_year, dtype: float64

In [66]:
df[:-1]

,a,b
0,1,4
1,2,5


In [71]:
import functools

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})

def drop_first(df):
    return df[1:]

def drop_last(df):
    return df[:-1]
    

functools.reduce(lambda x, f: f(x), [drop_first, drop_last], df)

,a,b
1,2,5
